In [0]:
%sql
CREATE OR REPLACE TABLE workspace.gold_serving.dimension_users
USING DELTA
AS
SELECT
  time,
  submit_id,
  submit_datetime,
  review_id,
  review_datetime,
  approval_id,
  approval_datetime,
  status
FROM workspace.gold_staging.stg_users;
